# Balancer Simulations

# The LP package

This is a model and set of simulations to analyse Balancer AMM pools from a liquidity provider point of view,  
using **historical on-chain data**.

You'll be able to analyse **pool share value** in the following scenarios:  
- C1 Analyse **pool share value over time** (incl. Impermanent Loss)  
- C2 Analyse pool share value **when add** (incl. slippage)
- C3 Analyse pool share value **when exit** (incl. slippage)

For more information on available packages, checkout the [Balancer Simulations documentation](xxxadd link).

## Table of content
here List and links to content (automatically?)

# A. System Context

### Value Drivers & Key Variables
Differential Syntax Diagram goes here

## Naming Convention

All code provided for this package is following a specific naming convention. You'll find and overview in the documentation. (xxx add link)


## List of State Variables  


In [ ]:
from pprint import pprint as pp
from model.genesis_states import initial_values
pp(initial_values)

## List of Parameters

In [ ]:
from decimal import Decimal
parameters = {
    'swap_fee': [Decimal(0.1)]
}
pp(parameters)

### System Mapping (Differential spec here? Or in later specifying simulations)
Show the (digital twin!) system in a viz language like
- Conceptual model
- Causal Loop
- Stock & Flow
...or several of them, tbd

### Mathematical Specification (Policy here? Or in later specifying simulations)
shows the Balancer AMM math (or provides a link),
add some more context, particularly
- what function is used where (reference to What-if-Matrix, perhaps add eg. "Spotprice" to What-if-Matrix) this reference of Math > Simulation would be ideal

### System Design (see above)
Show the (digital twin!) system in
- Differential Spec Syntax

# B. cadCAD Notebook

### 0. Dependencies
optionally add context to code below, and link "requirements.txt" 

In [ ]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

# C. Simulate: 
# Change of LP pool share value over time

### C1.1 Scenario

This simulation allows you to analyse the change of a liquidity provider’s pool share value over time.  
You can run various scenarios  
a) step into the shoes of a specific liquidity provider (should be the first transaction in your data set) and simulate your returns, potential losses or optimization strategies “what if I were this person” (100% overlap with on-chain data)  
b) define a new liquidity add (make artificial “JOIN” event, which didn’t exist in on-chain), and simulate your returns, potential losses or optimization strategies (based on on-chain transactions, deviations according to your liquidity add)  

### C1.2 State Variables


Define token symbol for better readability:

In [ ]:
#code to assign token symbol to transaction data (token smart contracts) 

Pull initial balances from data: (how to data parse in readme.file in data folder)

In [ ]:
# Date/block timestampStart
# Date/block timestampStop

import pprint
from model.genesis_states import initial_values
pp = pprint.PrettyPrinter(indent=4)

print('## State Variables')
print('# Pool')
gen_values = initial_values
pool = gen_values['pool']
pp.pprint(pool)
print('# External token values feeds')
token_values = gen_values['token_values']
pp.pprint(token_values)




### C1.3 System Parameters
(provides an overview on all parameters)

The system parameters we need to **define** for simulating this use case are:
- parameter description in human langage / parameter (code) / with value (code)

The following system parameters will be **swept** in the simulation:
- parameter description in human language / parameter (code) / values to sweep (code)



### C1.4 State Update Functions

Balancer Math implementation in python in file [balancer_math](./model/parts/balancer_math.py)

In regard to this simulation there are 3 types of action the users can do:

#### Add liquidity (join pool)

Transfer tokens into the pool proportionally to the tokens weights in it, and receive an amount of pool share tokens that represent the user's position that the pool will mint, adding to the total_pool_shares.

Balancer allows to add liqudity in 2 ways

- `s_join_pool`: Defining the amount of pool_share tokens the user wants out. This will calculate the amount of each token bound to the pool that will be pulled from the user's wallet, in proportion to the bound tokens weight in the pool.

- `s_join_swap_extern_amount_in`: Sending a determinate amount of 1 of the bound tokens. The amount of pool_share tokens out will be calculated and returned to the user. This operation acts like a swap as well, so swap_fees will be aplied.



#### Remove liquidity (exit pool)

Transfer tokens out from the pool to the user's ballance, proportionally to the tokens weights in it, sending the corresponding pool_share tokens back to the pull, which will burn them reducing total_pool_shares

Balancer allows to remove liqudity in 2 ways

- `s_exit_pool`: Defining the amount of pool_share tokens the user send to the pool and burn. This will calculate the amount of each token bound to the pool that will be sent to the user's wallet, in proportion to the bound tokens weight in the pool.

- `s_exit_swap_pool_amount_in`: Reclaiming a determinate amount of 1 of the bound tokens. The amount of pool_share tokens needed to reclaim this amount will be calculated and pulled from the user's balance and burnt. This operation acts like a swap as well, so swap_fees will be aplied.


#### Swap tokens (exchange one token for another)

- `s_swap_exact_amount_in`: Send one token to the pool, defining which token the user wants in return.



#### Simplifications in the model

Balancer has 2 variants of each method, one where you fix the amount of tokens in and the amount and a limit on the amount of tokens out or swap price, and one fixing the amount of tokens out and limits on the swap price or amount of tokens in.

The reason to this is that in Ethereum transactions can get mined before the one sent by the user, so pool state can change before the transaction is executed, resulting in trades not benefitial for the user. 

For this, the pool methods will check the user define limits and will make the transaction fail if is not within the user defined bounds. This way gas will be spent and lost, but unfavorable trades will not be made.

Since our simulation deals with sequential transactions, *this logic is not included and all methods will use math for known token amount in.*





Defined in [pool_state_updates.py](./model/parts/pool_state_updates.py)

### C1.5 Partial State Update Blocks


1. Parse action and update pool
2. Update external prices
3. Calculate metrics

The BPool smart contract logic is split in 2, the state update blocks 1 (apply BMath to update pool state) and 3 (use BMath to `get_spot_price` of the tokens after the trades, which is a system metric)



Defined in [partial_state_update_block.py](./model/partial_state_update_block.py)

In [ ]:
from model.partial_state_update_block import generate_partial_state_update_blocks

result = generate_partial_state_update_blocks('model/parts/actions_prices-WETH-DAI-0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a.json')
partial_state_update_blocks = result['partial_state_update_blocks']
pp.pprint(partial_state_update_blocks)

### C1.6 Configuration




In [ ]:
steps_number = result['steps_number']
print('# Steps ', steps_number)
sim_config = config_sim(
    {
        'N': 1,  # number of monte carlo runs
        'T': range(steps_number - 1),  # number of timesteps - 1267203 is last action timestep (timestamp - initial timestamp)
        'M': parameters,  # simulation parameters
    }
)

### C1.7 Execution

In [ ]:
from model.genesis_states import initial_values

from model.sim_runner import *

df = run(initial_values, partial_state_update_blocks, sim_config)


### C1.8 Simulation Output Preparation

In [ ]:
from model.parts.utils import post_processing

p_df = post_processing(df)
#p_df.tail(10)

### C1.9 Simulation Outcome & Conclusion

Note: 
we use the following color code for plots  
a) token balances = green  
b) USD values = orange  
c) pool shares (BPT balances) = blue  

#### C.1.9.1 Compare Token Balances

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

In [ ]:
#USE VALUES DIRECTLY FROM DATAFRAME ZERO (p_df)

x = p_df['timestep'] #change to datetime, solve processing issue
y1 = p_df['token_weth_balance']
y2 = p_df['token_dai_balance']
y3 = p_df['pool_shares']

plt.figure()
fig, axs = plt.subplots(figsize=(20,10), dpi=300, ncols=1, nrows=3, constrained_layout=True)
fig.suptitle('Balances', fontsize=24)
axs[0].plot(x, y1, linewidth=1, marker=".", color="#3C7E6F")
axs[0].set_title('Token WETH Balance', fontsize=16)
axs[0].set_xlabel('timestep', fontsize=16)
axs[0].set_ylabel('token_weth_balance', fontsize=16)

axs[1].plot(x, y2, linewidth=1, marker=".", color="#53992C")
axs[1].set_title('Token DAI Balance', fontsize=16)
axs[1].set_xlabel('timestep', fontsize=16)
axs[1].set_ylabel('token_dai_balance', fontsize=16)

axs[2].plot(x, y3, linewidth=1, marker=".", color="#5CB1EC")
axs[2].set_title('Pool Share Balance', fontsize=16)
axs[2].set_xlabel('timestep', fontsize=16)
axs[2].set_ylabel('pool_shares', fontsize=16)


plt.show()

### Observations:

1.Observe arbitrage trading behavior
- there are periods diplaying either more interest in ETH or DAI (see rough curve direction upwards, downwards)
- how does that correlate with arbitrage gaps/arbitrage trading (assumption: a sequence of trades pushing the curve in the same direction are trades until an arbitrage gap is closed
- in future versions we might be able to classify arbitrage trades. What if it turns out that 90% of trades on typical AMMs are arbitrage trades? Isn't this a huge value extract, too big to ignore? DAMMs come to the rescue :)

In [ ]:
#ANALYSE TVL
#CREATE CUSTOMIZED DATAFRAME FROM DATAFRAME ZERO (p_df)

tvl_p_df = pd.DataFrame(columns=['total_token_value_TVL', 'timestep'])
tvl_p_df['timestep'] = p_df['timestep'] 

#provide list of tokens (can perhaps be refactored to pull automatically from genesis_state.py?)
tlist = ('dai','weth')

# append existing columns (balance+price), and create new column for values
for i in tlist:
    tvl_p_df[f'token_{i}_balance'] = p_df[f'token_{i}_balance'] 
    tvl_p_df[f'token_{i}_price'] = p_df[f'token_{i}_value'] 
    tvl_p_df[f'token_{i}_value'] = tvl_p_df[f'token_{i}_balance']*tvl_p_df[f'token_{i}_price']

# calculate TVL
for i in tlist: #needs refactoring, so that we not only pick a+b but all tokens (up to 8) values
    a = tvl_p_df[f'token_{tlist[0]}_value']
    b = tvl_p_df[f'token_{tlist[1]}_value']
    tvl_p_df['total_token_value_TVL'] = a + b
tvl_p_df.tail(10) 

In [ ]:
x = tvl_p_df['timestep'] #change to datetime, solve processing issue
y1 = tvl_p_df['total_token_value_TVL']
y2 = tvl_p_df['token_weth_value']
y3 = tvl_p_df['token_dai_value']

plt.figure()
fig, axs = plt.subplots(figsize=(20,10), dpi=300, ncols=1, nrows=3, constrained_layout=True)
fig.suptitle('TVL', fontsize=24)
axs[0].plot(x, y1, linewidth=1, marker=".", color="#3C7E6F")
axs[0].set_title('Token Token Value TVL', fontsize=16)
axs[0].set_xlabel('timestep', fontsize=16)
axs[0].set_ylabel('total_token_value_TVL', fontsize=16)

axs[1].plot(x, y2, linewidth=1, marker=".", color="#53992C")
axs[1].set_title('Token WETH Value', fontsize=16)
axs[1].set_xlabel('timestep', fontsize=16)
axs[1].set_ylabel('token_dai_value', fontsize=16)

axs[2].plot(x, y3, linewidth=1, marker=".", color="#5CB1EC")
axs[2].set_title('Token DAI Value', fontsize=16)
axs[2].set_xlabel('timestep', fontsize=16)
axs[2].set_ylabel('token_dai_value', fontsize=16)


plt.show()

# System Validation and Limitations

- document the steps taken to validate if the model reflects real Balancer AMM properly (Did we build the right model?)
- document the steps taken to verify if the model creates reliable results (Did we build the model right?)

### Notes (Draft!)
**a) BMath Calculations:**  
Our goal is to implement the BMath calculations in this Python model in a way that it replicates *exactly* the calculation results in an EVM.

We've verified the model with a series of tests:
- create tests using balancer's smart contract repos
- generate a pool contract in a local EVM, do a swap or whatever operation, 
- put those input outputs as a test in python, port the code, test to see if the results match

**b) external USD price feed** 
- in this simulation we're using historical USD prices from xxx (source)  
- to map blocks and transactions we've ... (how we parsed USD price feed)

**c) Our simulation does not include:**  
- gas prices or add_fees when adding liquidity  
- 

**Results: (summarize)**

(Notes for ourselves:
- assertAlmostEqual takes 7 decimal places for comparision, sometimes we had to set 5 decimal places for the test to pass
- we could publish the EVM tests  as companion in the docs later but right now is very rough code
- to run the test go to your virtual env, instlal requirements with pip, then run pytest
- they should pass
- when everything is tested and works as the contracts, we could move on to cadCAD stuff

According to Balancer.finance documentation "The formulas are sufficient to describe the functional specification, but they are not straightforward to implement for the EVM, in part due to a lack of mature fixed-point math libraries." (https://docs.balancer.finance/core-concepts/protocol/index))

# E. Comments

closing comments if appropriate, and links to other notebooks/other use cases